<a href="https://colab.research.google.com/github/fullstackdata/DSND_Term2/blob/master/llamaindex_quantized_offline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate bitsandbytes sentence-transformers llama-index llama-index-embeddings-huggingface datasets llama-index-llms-huggingface

In [ ]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex, Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.huggingface import HuggingFaceLLM
import torch

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in DeployedModel has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_kwargs" in HuggingFaceLLM has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPI has co

In [ ]:
# !wget https://huggingface.co/NousResearch/Meta-Llama-3.1-8B/resolve/main/model-00001-of-00004.safetensors
# !wget https://huggingface.co/NousResearch/Meta-Llama-3.1-8B/resolve/main/model-00002-of-00004.safetensors
# !wget https://huggingface.co/NousResearch/Meta-Llama-3.1-8B/resolve/main/model-00003-of-00004.safetensors
# !wget https://huggingface.co/NousResearch/Meta-Llama-3.1-8B/resolve/main/model-00004-of-00004.safetensors

#### Load data

In [ ]:
from datasets import load_dataset
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

wkds = load_dataset("microsoft/wiki_qa", split="train", streaming=True)
wkitr = iter(wkds)
recs = []

for i in range(10):
  recs.append(next(wkitr))

processed_data_list = [rec['answer'] for rec in recs]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

#### Chunk, embed index and store

In [ ]:


bge_name = "BAAI/bge-small-en-v1.5"

bge_cache_path="/root/.cache/huggingface/hub/models--BAAI--bge-small-en-v1.5/snapshots/5c38ec7c405ec4b44b94cc5a9bb96e735b38267a"
Settings.embed_model = HuggingFaceEmbedding(model_name=bge_name)


from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(bge_name)

splitter = SentenceSplitter(
    chunk_size=256,
    chunk_overlap=5,
    tokenizer=tokenizer
)

lngdocs = [Document(text=wktxt) for wktxt in processed_data_list]
nodes = splitter.get_nodes_from_documents(lngdocs)

index = VectorStoreIndex(nodes)

query_str = "give me an example of a circular motion?"

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

#### Load quantized LLM (llama3.1 - 8B )

In [ ]:
import torch
# llama_4_bit = "/Users/chandana/.cache/lm-studio/models/mlx-community/Llama-3.2-3B-Instruct-4bit"
# gguf = "NousResearch/Llama-3.2-1B"
llama318B = "/content/NousResearch/Meta-Llama-3.1-8B"

model_used=llama318B

from transformers import AutoTokenizer

from transformers import BitsAndBytesConfig


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    # quant_method =
)

tokenizer = AutoTokenizer.from_pretrained(
    model_used,
    trust_remote_code=True
)

stopping_ids = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>"),
]

llm = HuggingFaceLLM(
    model_name=model_used,
    model_kwargs={
        "torch_dtype": torch.bfloat16,  # comment this line and uncomment below to use 4bit
        "quantization_config": quantization_config
    },
    generate_kwargs={
        "do_sample": True,
        "temperature": 0.6,
        "top_p": 0.9,
    },
    tokenizer_name=model_used,
    stopping_ids=stopping_ids,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

#### Query and generate response

In [ ]:
query_str = "give me an example of a circular motion?"

query_engine = index.as_query_engine(similarity_top_k=2, llm=llm)# use this for testingvector_retriever = index.as_retriever(similarity_top_k=2)

response = query_engine.query(query_str)
print(response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


 An artificial satellite orbiting the Earth at constant height, a stone which is tied to a rope and is being swung in circles, a car turning through a curve in a race track, an electron moving perpendicular to a uniform magnetic field, and a gear turning inside a mechanism.
---------------------

